In [28]:
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient

from time import sleep

import pandas as pd
import numpy as np

Through inspecting the page it looks like I can get all the data I need here.  I am going to get the sub page information as well but for now just making sure I can get everything.

In [29]:
url = 'https://www.pro-football-reference.com/years/2019/receiving.htm#'

page = requests.get(url)
page

<Response [200]>

In [30]:
page.status_code

200

In [31]:
soup = BeautifulSoup(page.text)

After looking at the page it looks like there is only one `tbody` and that is where the data I want is.  So after checking that this is true we can move on.

In [32]:
len(soup.find_all('tbody'))

1

So a given season page seems to only have one table so we are good.  Now to check what a row looks like and make sure the data I want is in there.  I also need to find the url link for the player pages as I want those as well.

Finally it looks like there are rows with missing data (for viewing ease) I will need to make sure to skip.

In [33]:
table = soup.find_all('tbody')[0]

In [34]:
row = table.find_all('tr')[1]

In [35]:
len(table.find_all('tr'))

507

In [36]:
row.find_all('td')

[<td class="left" csk="McCaffrey,Christian" data-append-csv="McCaCh01" data-stat="player"><a href="/players/M/McCaCh01.htm">Christian McCaffrey </a>*+</td>,
 <td class="left" data-stat="team"><a href="/teams/car/2019.htm" title="Carolina Panthers">CAR</a></td>,
 <td class="right" data-stat="age">23</td>,
 <td class="left" data-stat="pos">RB</td>,
 <td class="right" data-stat="g">16</td>,
 <td class="right" data-stat="gs">16</td>,
 <td class="right" data-stat="targets">142</td>,
 <td class="right" data-stat="rec">116</td>,
 <td class="right" data-stat="catch_pct">81.7%</td>,
 <td class="right" data-stat="rec_yds">1005</td>,
 <td class="right" data-stat="rec_yds_per_rec">8.7</td>,
 <td class="right" data-stat="rec_td">4</td>,
 <td class="right" data-stat="rec_first_down">58</td>,
 <td class="right" data-stat="rec_long">28</td>,
 <td class="right" data-stat="rec_yds_per_tgt">7.1</td>,
 <td class="right" data-stat="rec_per_g">7.3</td>,
 <td class="right" data-stat="rec_yds_per_g">62.8</td>

We seem to have the right amount of row data and each row looks to have the data I want.  So I should be good to move to the next step.  One thing to note is that we have sub pages to scrape.

In [37]:
col_one = row.find('td')

In [38]:
player = col_one.find('a')

In [39]:
player.get_attribute_list('href')[0]

'/players/M/McCaCh01.htm'

In [40]:
col_one.get('data-stat')

'player'

In [2]:
position = 'receiving'

Putting it all together

In [4]:
client = MongoClient()
db = client['nfl']
season_collection = db['season_raw']
player_collection = db['player_raw']

for year in range(2005, 2006):
    url = 'https://www.pro-football-reference.com/years/{}/{}.htm#'.format(year, position)
    print(url)
    
    page = requests.get(url)
    
    # save the raw HTML so I never have to scrape again (just reparese)
    season_collection.insert_one({'year':year,'position':position, 'html':page.text})
    
    soup = BeautifulSoup(page.text)
    
    # Get all player sub pages:
    table = soup.find('tbody')
    for row in table.find_all('tr'):
        col_one = row.find('td')
        if col_one == None:
            continue
        player = col_one.find('a')
        url_player = player.get_attribute_list('href')[0]
        
        sub_url = 'https://www.pro-football-reference.com{}'.format(url_player)
        sub_page = requests.get(sub_url)
        
        wtf.append([sub_page.status_code, url_player])
        
        player_collection.insert_one({'url':sub_url, 
                                      'player': url_player.split('/')[-1],
                                      'player_url': url_player,
                                      'html':sub_page.text,
                                      'position':position,
                                      'year':year})
        sleep(3)


https://www.pro-football-reference.com/years/2005/receiving.htm#
